# AH Bonnen parser

In [24]:
import os
import re
import io
import PyPDF2
from PIL import Image
import pandas as pd
from pdf2image import convert_from_path
import pytesseract

import warnings
warnings.filterwarnings("ignore")

In [25]:
def receipt_renamer(folder_path):
    """
    Function renames the download ah receipts to a more sortable name
    Input: A path to a folder with ah receipt pdfs. 
    """

    # Get a list of all the files in the directory
    files = os.listdir(folder_path)
    # Loop through each file in the directory
    for file in files:
        # Use regular expressions to extract the date from the filename
        match = re.search(r'\d{2}-\d{2}-\d{2}', file)
        if match:
            # Extract the date from the match object
            date = match.group()
            # Convert the date to the desired format (YYYY-MM-DD)
            new_name = re.sub(r'(\d{2})-(\d{2})-(\d{2})', r'20\3-\2-\1', date) + '_AH_kassabon'
            # Rename the file with the new name
            try:
                os.rename(os.path.join(folder_path, file), os.path.join(folder_path, new_name + '.pdf'))
            except Exception as E:
                print(E)
    print('succesfully renamed all files')

In [26]:
def pdf_to_img(pdf_path, pdf_img_path):

    # Get a list of all the files in the directory
    files = os.listdir(pdf_path)
    # Loop through each file in the directory
    for pdf_file in files:

        full_path = os.path.join(pdf_path, pdf_file)
        # Convert the first page of the PDF file to a PIL image
        images = convert_from_path(full_path, first_page=1, last_page=1)
        image = images[0]
        # obtain the pdf name to name the image.
        pdf_file = pdf_file.replace('.pdf','.jpg')
        # Save the image to a file
        image.save(pdf_img_path + pdf_file)
        
    print('succesfully converted all pdf files to images')

In [27]:
#pdf_to_img('ah_bonnen_pdf', 'ah_bonnen_img\\')

In [28]:
def text_to_df(text, img_file):

    temp_df = pd.DataFrame(columns=['Amount','Product','Price'])
    try:
        text_without_start = text.split('BONUSKAART xx9644', 1)[1]
        text_without_end = text_without_start.split('UW VOORDEEL',1)[0]
        lines = text_without_end.split("\n")
    except Exception as ex:
        print(img_file, ' did not work because', ex)
        return temp_df

    match = re.search(r"\d{1,2}-\d{1,2}-\d{4}", text)
    if match:
        date = match.group()
        date_without_time = re.sub(r"\s+\d{1,2}:\d{1,2}\s*", " ", date)

    for line in lines:
        # Create an I that decides which part should be the amount.
        i = -1
        if line:
            parts = line.split()
            amount = parts[0]

            # Ignore the bonus box lines.
            if 'BBOX' in amount or 'BONUS' in amount:
                continue
            # The bonus and bonux box lines are the last on receipt.
            # Therefore they have to be ignored to get the price.
            while parts[i] == 'B' or parts[i] == 'BB':
                i -=1
            
            # If the amount of the product is expressed in KG, we want to grab the kg price
            # And remove KG from the line
            if 'KG' in amount:
                i -=1
                product_name = " ".join(parts[1:i])
                amount = amount.replace('KG','')
            else:
                product_name = " ".join(parts[1:i])

            # Assign price and make it a float
            # Also split price if the value is not right
            price = parts[i]
            price = price.replace(',', '.')
            try:
                if float(price) > 100:
                    price = price[:1] + '.' + price[1:]
                price = float(price)
            except Exception as ex2:
                print('Encounterd: ', ex2, 'For ', product_name, 'In', date_without_time)
                price = 999

            temp_df.loc[len(temp_df)] = [amount, product_name, price]

    temp_df['Amount'] = pd.to_numeric(temp_df['Amount'], errors='coerce').fillna(1)
    temp_df['Date'] = pd.to_datetime(date_without_time, format='%d-%m-%Y').date()
    # Multiple the amount bought by the price to get the right price
    # Known issue: Messes up subtotaal.
    # If price is not a float it will return None
    temp_df['KG_Price'] = temp_df.apply(lambda x: round(x['Price'] * x['Amount'], 2) if isinstance(x['Price'], float) else None, axis=1)
    temp_df['indv_price'] = temp_df.apply(lambda x: round(x['Price'] / x['Amount'], 2) if isinstance(x['Price'], float) else None, axis=1)
    return temp_df 

In [29]:
def text_from_img(pdf_img_path):

    df = pd.DataFrame(columns=['Amount','Product','Price','Date'])

    # Get a list of all the files in the directory
    files = os.listdir(pdf_img_path)
    # Loop through each file in the directory
    for img_file in files:

        full_path = os.path.join(pdf_img_path, img_file)
        image = Image.open(full_path)
        # Convert the image to grayscale
        image = image.convert('L')
        # Process the image with pytesseract
        text = pytesseract.image_to_string(image)
        temp_df = text_to_df(text, img_file)
        df = pd.concat([df, temp_df]).reset_index(drop=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values(by='Date')
    return df


In [30]:
img_path = 'C:\\Users\\rie12\\Documents\\GitHub\\AH_kassabonnen\\ah_bonnen_img'

In [31]:
test = text_from_img(img_path)

2001-03-22_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string to float: '=' For  GOUDSALAMI In 1-8-2022
Encounterd:  could not convert string to float: 'GOUDSALAMI' For   In 8-11-2021
Encounterd:  could not convert string to float: 'LAS' For  Gl PENNE In 13-11-2021
Encounterd:  could not convert string to float: '—' For  AH ROAST HAM In 15-11-2021
Encounterd:  could not convert string to float: '-' For  GOUDSALAMI In 5-12-2021
Encounterd:  could not convert string to float: 'i' For  KIPFILET In 9-12-2021
2022-01-07_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string to float: 'ia' For  AH YORKHAM In 11-1-2022
Encounterd:  could not convert string to float: '35%' For  KL SCHIJFJES 0,65 In 19-1-2022
Encounterd:  could not convert string to float: 'Ly' For  DONUT SUIKER In 19-2-2022
2022-05-04_AH_kassabon.jpg  did not work because list index out of range
Encounterd:  could not convert string t

In [32]:
test.to_csv('bonnetjes_v1.csv', index=False)

# AH Bonnen grafieken

Wat wil ik laten zien:

* Totale Prijs verschil over tijd
* Meest gekochte producten
* Voor meest gekochte producten het verschil over tijd

In [9]:
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

In [33]:
test = pd.read_csv('bonnetjes_v1.csv')

In [34]:
test

,Amount,Product,Price,Date,KG_Price,indv_price
0,5.0,SUBTOTAAL,9.54,2021-10-11,47.70,1.91
1,1.0,PASTASAUS,1.79,2021-10-11,1.79,1.79
2,1.0,AH SPAGHETTI,2.25,2021-10-11,2.25,2.25
3,1.0,ZAANSE HOEVE,1.49,2021-10-11,1.49,1.49
4,1.0,QUAKER CRUES,2.99,2021-10-11,2.99,2.99
...,...,...,...,...,...,...
1977,1.0,DESPERADOS,9.79,2023-04-30,9.79,9.79
1978,1.0,OETKER PIZZA,4.19,2023-04-30,4.19,4.19
1979,1.0,ZAANS BRUIN,1.09,2023-04-30,1.09,1.09
1980,1.0,LAY'S OVEN B,2.29,2023-04-30,2.29,2.29


In [6]:
# Totale prijs df maken.
test['Price'] = pd.to_numeric(test['Price'])
total_df = test[test['Product'] == 'SUBTOTAAL']

In [36]:
# Calculate rolling mean for Price column
total_df['Rolling Price'] = total_df['Price'].rolling(window=10).mean()
total_df['Rolling Amount'] = total_df['Amount'].rolling(window=10).mean()

In [58]:
total_df

,Amount,Product,Price,Date,Price_Amount,divided_price,Rolling Mean,Rolling Price,Rolling Amount
0,5.0,SUBTOTAAL,9.54,2021-10-11,47.70,1.908000,NaN,NaN,NaN
8,9.0,SUBTOTAAL,14.75,2021-10-12,132.75,1.638889,NaN,NaN,NaN
17,8.0,SUBTOTAAL,17.76,2021-10-13,142.08,2.220000,NaN,NaN,NaN
31,5.0,SUBTOTAAL,7.64,2021-10-25,38.20,1.528000,NaN,NaN,NaN
32,7.0,SUBTOTAAL,10.59,2021-10-27,74.13,1.512857,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1924,12.0,SUBTOTAAL,40.36,2023-04-13,484.32,3.363333,26.824,26.824,10.7
1936,13.0,SUBTOTAAL,37.03,2023-04-17,481.39,2.848462,28.617,28.617,11.2
1951,11.0,SUBTOTAAL,21.05,2023-04-19,231.55,1.913636,26.708,26.708,10.4
1962,3.0,SUBTOTAAL,7.97,2023-04-21,23.91,2.656667,24.262,24.262,8.9


In [72]:
line1 = go.Scatter(x=total_df['Date'], y=total_df['Price'], name='Price over Time', opacity=0.3, line=dict(color='red'))
line2 = go.Scatter(x=total_df['Date'], y=total_df['Rolling Price'], name='10 visits rolling mean', line=dict(color='red'))
line3 = go.Scatter(x=total_df['Date'], y=total_df['Amount'], name='Amount over Time', opacity=0.3, line=dict(color='blue'))
line4 = go.Scatter(x=total_df['Date'], y=total_df['Rolling Amount'], name='10 visits rolling amount', line=dict(color='blue'))
data = [line1, line2, line3, line4]
fig = go.Figure(data=data)

fig.update_layout(title_text = """The total amount and price of items bought""" ,
                  xaxis_title = "Months", 
                  yaxis_title = 'Price in € & Amount')
fig.show()

In [60]:
total_df['Rolling Price'].min()

13.599

In [69]:
first_valid_index = total_df['Rolling Price'].first_valid_index()
first_rolling_price = total_df['Rolling Price'].loc[first_valid_index]
last_rolling_price = total_df['Rolling Price'].iloc[-1]
percent_diff = round(((last_rolling_price - first_rolling_price) / first_rolling_price) * 100, 2)

print(' First 10 point average =', round(first_rolling_price, 2), '\n', 'Last 10 point average =', round(last_rolling_price, 2))
print(" Percentage difference in rolling price:", percent_diff, '%')

 First 10 point average = 13.6 
 Last 10 point average = 26.05
 Percentage difference in rolling price: 91.58 %


In [70]:
first_valid_index_Amount = total_df['Rolling Amount'].first_valid_index()
first_rolling_Amount = total_df['Rolling Amount'].loc[first_valid_index_Amount]
last_rolling_Amount = total_df['Rolling Amount'].iloc[-1]
percent_diff_Amount = round(((last_rolling_Amount - first_rolling_Amount) / first_rolling_Amount) * 100, 2)

print(' First 10 point average =', round(first_rolling_Amount, 2), '\n', 'Last 10 point average =', round(last_rolling_Amount, 2))
print(" Percentage difference in rolling price:", percent_diff_Amount, '%')

 First 10 point average = 7.9 
 Last 10 point average = 9.9
 Percentage difference in rolling price: 25.32 %


## Meest voorkomende producten

In [11]:
most_common = test.groupby('Product')['Product'].count()

In [12]:
df_grouped = test.groupby('Product').agg({'Amount': 'sum', 'Product': 'size'})
df_grouped = df_grouped.rename(columns={'Product':'Count'})
df_grouped = df_grouped.reset_index(drop=False)
df_grouped = df_grouped.sort_values(by='Count', ascending=False)

In [13]:
df_grouped.head(30)

,Product,Amount,Count
458,SUBTOTAAL,1832.000,171
362,OETKER PIZZA,63.000,63
393,PICKWICK,45.000,45
263,HK CROISSANT,42.000,42
516,ZAANS BRUIN,39.000,39
470,TIJGER WIT,36.000,36
144,BANANEN,26.697,30
328,LAY'S OVEN B,30.000,30
189,CUP A SOUP,26.000,26
197,DORITOS,25.000,25


# Individual product graphs

In [39]:
def graph_maker(df, product_name, rolling_amount=5):
    df = df[df['Product'] == product_name]
    # Calculate rolling mean for Price column
    df['Rolling Mean'] = df['Price'].rolling(window=rolling_amount).mean()
    fig = px.bar(df, x='Date', y=['Price', 'Rolling Mean'], title=f'Price for {product_name} over Time')
    fig.update_traces(line=dict(width=2)) # Adjust line width
    return fig

In [40]:
graph_maker(test, 'DORITOS')

ValueError: Invalid property specified for object of type plotly.graph_objs.Bar: 'line'

Did you mean "base"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        base
            Sets where the bar base is drawn (in position axis
            units). In "stack" or "relative" barmode, traces that
            set "base" will be excluded and drawn in "overlay" mode
            instead.
        basesrc
            Sets the source reference on Chart Studio Cloud for
            base .
        cliponaxis
            Determines whether the text nodes are clipped about the
            subplot axes. To show the text nodes above axis lines
            and tick labels, make sure to set `xaxis.layer` and
            `yaxis.layer` to *below traces*.
        constrainte